#Exercise No3: Thực hành Sử dụng Selenium để cralw dữ liệu BCTC về và lưu thành Excel
Nguồn tham khảo: https://www.youtube.com/watch?v=k9immzcwtHg

##1. Link download webdriver
Chrome: https://chromedriver.chromium.org/downloads
        https://googlechromelabs.github.io/chrome-for-testing/#stable
Firefox: https://github.com/mozilla/geckodriver/releases

##2. Cài đặt và import thư viện liên quan đến selenium và firefox



In [1]:
pip install selenium

  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.5 MB 1.7 MB/s eta 0:00:06
    --------------------------------------- 0.1/9.5 MB 1.4 MB/s eta 0:00:07
    --------------------------------------- 0.2/9.5 MB 1.6 MB/s eta 0:00:06
   - -------------------------------------- 0.3/9.5 MB 1.6 MB/s eta 0:00:06
   - -------------------------------------- 0.4/9.5 MB 1.6 MB/s eta 0:00:06
   - -------------------------------------- 0.5/9.5 MB 1.7 MB/s eta 0:00:06
   -- ------------------------------------- 0.5/9.5 MB 1.7 MB/s eta 0:00:06
   -- ------------------------------------- 0.6/9.5 MB 1.6 MB/s eta 0:00:06
   -- ------------------------------------- 0.7/9.5 MB 1.6 MB/s eta 0:00:06
   --- ------------------------------------ 0.8/9.5 MB 1.7 MB/s eta 0:00:06
   --- ------------------------------------ 0.9/9.5 MB 1.8 MB/s eta 0:00:05
   ---- ------------

In [2]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas

In [3]:
pip install xlsxwriter

   ---------------------------------------- 0.0/159.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/159.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/159.9 kB ? eta -:--:--
   ------- ------------------------------- 30.7/159.9 kB 325.1 kB/s eta 0:00:01
   ------- ------------------------------- 30.7/159.9 kB 325.1 kB/s eta 0:00:01
   ------- ------------------------------- 30.7/159.9 kB 325.1 kB/s eta 0:00:01
   ------- ------------------------------- 30.7/159.9 kB 325.1 kB/s eta 0:00:01
   ------- ------------------------------- 30.7/159.9 kB 325.1 kB/s eta 0:00:01
   --------- ----------------------------- 41.0/159.9 kB 103.4 kB/s eta 0:00:02
   --------- ----------------------------- 41.0/159.9 kB 103.4 kB/s eta 0:00:02
   --------- ----------------------------- 41.0/159.9 kB 103.4 kB/s eta 0:00:02
   --------- ----------------------------- 41.0/159.9 kB 103.4 kB/s eta 0:00:02
   -------------- ------------------------ 61.4/159.9 kB 102.4 kB

In [1]:
#Import thư viện
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.firefox.service import Service as FireFoxService
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import pandas as pd

In [6]:
# Chrome Driver Folder: D:\Setups\chrome-win64\chrome.exe.exe
#chromeService = ChromeService(executable_path=r"D:\Setups\chrome-win64\chrome.exe.exe")
#As of Selenium 4.6, you no longer need to manage your browser drivers. Selenium Manager was added and will automatically download, configure, and use the appropriate browser driver. 
#https://stackoverflow.com/questions/78349830/chromedriver-124-is-only-available-for-beta-and-that-is-giving-error-unable-to
driver_manual = webdriver.Chrome()
driver_manual.get("https://cafef.vn/")

In [8]:
# Firefox Driver Folder: D:\Setups\geckodriver\geckodriver.exe
firefoxService = FireFoxService(GeckoDriverManager().install())
driver_auto = webdriver.Firefox(service=firefoxService)
driver_auto.get("https://cafef.vn/")

URL báo cáo tài chính từ Cafef.vn
Ví dụ FPT:
- Bảng cân đối kế toán: https://s.cafef.vn/bao-cao-tai-chinh/fpt/bsheet/2024/1/0/0/bao-cao-tai-chinh-.chn
- Báo cáo KQKD: https://s.cafef.vn/bao-cao-tai-chinh/fpt/incsta/2024/1/0/0/bao-cao-tai-chinh-.chn
- Lưu chuyển tiền tệ: https://s.cafef.vn/bao-cao-tai-chinh/fpt/cashflow/2024/1/0/0/ket-qua-hoat-dong-kinh-doanh-.chn


In [9]:
driver_auto = webdriver.Chrome()
driver_auto.get("https://s.cafef.vn/bao-cao-tai-chinh/fpt/bsheet/2024/1/0/0/bao-cao-tai-chinh-.chn")
driver_auto.implicitly_wait(10)


Sau khi trang web mở thì bấm phím F12 trên trình duyệt để vào chế độ Developer.
Tìm đến cột đầu tiên của bảng báo cáo, right click vào và chọn Copy/Copy XPath, ví dụ được giá trị như dưới:

//*[@id="tblGridData"]/tbody/tr/td[2]

In [3]:
#Định nghĩa functions
def get_elements_value(items, convert_numeric=True):
    _ls = []
    if convert_numeric == False:
        for x in items:
            _ls.append(x.text)
    else:            
        for x in items:
            #print(x.text)
            val = x.text.strip().replace(',','')
            _ls.append(val)
    return _ls

def process_column(driver, col):
    xpath = f"//*[@id='tableContent']/tbody/tr/td[{col}]"
    rows = driver.find_elements(By.XPATH, xpath)
    return rows

def get_year_list(current_year, num):
    years = []
    while num > 0:
        years.append(current_year)
        num -= 1
    return years

def get_quarter_list(current_year, current_quarter, num):
    quarters = []
    while num > 0:
        quarters.append(f"Quý {current_quarter}-{current_year}")
        if current_quarter == 1:
            current_quarter = 4
            current_year -= 1
        else:
            current_quarter -= 1
        num -= 1
    return quarters



In [23]:
import datetime
import xlsxwriter
import os
current_datetime = datetime.datetime.now()

#tickers = ['FPT', 'PNJ', 'REE', 'DGC', 'KDH', 'HAX', 'TCH', 'HAG']
tickers = ['FPT']
reports = ['bsheet', 'incsta', 'cashflow']
report_times = [0, 1] #0: year, 1: quarter
#report_times = [0] #0: year, 1: quarter
report_folder = "C:/Users/hoant/OneDrive/Learning/IT/Python/100DaysOfCode/Reports/"
DATA_COLUMNS = 6
CURRENT_YEAR = current_datetime.year
CURRENT_MONTH = current_datetime.month
CURRENT_QUARTER = ((CURRENT_MONTH - 1) // 3) + 1

year_list = get_year_list(CURRENT_YEAR, DATA_COLUMNS)
quarter_list = get_quarter_list(CURRENT_YEAR, CURRENT_QUARTER, DATA_COLUMNS)
driver_auto = webdriver.Chrome()
for ticker in tickers:
    for report in reports:
        for time in report_times:
            #Target year
            y = CURRENT_YEAR
            if time == 0:
                y -= 1

            q = CURRENT_QUARTER          
            count_columns = 0
            data = {}
            criteria_names = {}
            while count_columns < DATA_COLUMNS:
                if time == 0:
                    url = f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/{report}/{y}/0/0/0/bao-cao-tai-chinh-.chn"
                else:
                    url = f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/{report}/{y}/{q}/0/0/bao-cao-tai-chinh-.chn"
                print(url)
                driver_auto.get(url)
                driver_auto.implicitly_wait(10)

                #data = {}
                #criteria_names = {}
                if 'criteria' not in criteria_names:
                    #//*[@id="tableContent"]/tbody/tr[1]/td[1]
                    #name_elements = driver_auto.find_elements(By.XPATH, "//*[@id='tableContent']/tbody/tr/td[1]")
                    name_elements = process_column(driver_auto, 1)
                    criteria_names['criteria'] = get_elements_value(name_elements, False)
                print(criteria_names)

                data_elements = driver_auto.find_elements(By.XPATH, "//*[@id='tblGridData']/tbody/tr/td")
                #print(quarter_elements[0].text)

                index_cols = {}
                i = 1
                for item in data_elements:
                    str_item = item.text.strip() 
                    index_cols[i] = str_item
                    i += 1
                
                index_cols.pop(6)
                index_cols.pop(1)
                index_cols = dict(sorted(index_cols.items(), reverse=True))

                for key in index_cols:
                    items = process_column(driver_auto, key)
                    data[index_cols[key]] = get_elements_value(items)
                    count_columns += 1
                    if time == 0:
                        y -= 1
                    else:
                        if q == 1:
                            q = 4
                            y -= 1
                        else:
                            q -= 1

                    if count_columns == DATA_COLUMNS:
                        break
                print(f"{ticker}: Report: {report}, Time: {time}, Year: {y}, Quarter: {q}")
                

            #Merge 2 dictionaries to one, then export to excel
            data = criteria_names | data
            
            #print(data)
            df = pd.DataFrame(data)
            print(df)
            save_as = f"{ticker}_{report}_{time}.xlsx"
            writer = pd.ExcelWriter(report_folder + save_as, engine='xlsxwriter')
            df.to_excel(writer, sheet_name=report, index=False)
            writer.close()
            #df
driver_auto.quit()
#os.startfile()

https://s.cafef.vn/bao-cao-tai-chinh/FPT/bsheet/2023/0/0/0/bao-cao-tai-chinh-.chn
{'criteria': ['TÀI SẢN', 'A- TÀI SẢN NGẮN HẠN', 'I. Tiền và các khoản tương đương tiền', '1. Tiền', '2. Các khoản tương đương tiền', 'II. Các khoản đầu tư tài chính ngắn hạn', '1. Chứng khoán kinh doanh', '2. Dự phòng giảm giá chứng khoán kinh doanh', '3. Đầu tư nắm giữ đến ngày đáo hạn', 'III. Các khoản phải thu ngắn hạn', '1. Phải thu ngắn hạn của khách hàng', '2. Trả trước cho người bán ngắn hạn', '3. Phải thu nội bộ ngắn hạn', '4. Phải thu theo tiến độ kế hoạch hợp đồng xây dựng', '5. Phải thu về cho vay ngắn hạn', '6. Phải thu ngắn hạn khác', '7. Dự phòng phải thu ngắn hạn khó đòi', '8. Tài sản Thiếu chờ xử lý', 'IV. Hàng tồn kho', '1. Hàng tồn kho', '2. Dự phòng giảm giá hàng tồn kho', 'V.Tài sản ngắn hạn khác', '1. Chi phí trả trước ngắn hạn', '2. Thuế GTGT được khấu trừ', '3. Thuế và các khoản khác phải thu Nhà nước', '4. Giao dịch mua bán lại trái phiếu Chính phủ', '5. Tài sản ngắn hạn khác', 'B.